In [1]:
import os
import random
import shutil
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Add, Activation, GlobalAveragePooling2D, Input
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
from PIL import Image

In [2]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
rar_path = '/content/drive/My Drive/CUDA-dataset/TrashNet_augmentedx2.rar'
extract_path = '/content/dataset'

os.makedirs(extract_path, exist_ok=True)

!unrar x -o+ "{rar_path}" "{extract_path}"

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,172_aug.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,173.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,173_aug.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,174.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,174_aug.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,175.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,175_aug.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,176.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,176_aug.jpg      68%  OK 
Extracting  /content/dataset/TrashNet_augmentedx2/plastic/PET1,177.jp

In [4]:
#shutil.rmtree('dataset/TrashNet/train')
#shutil.rmtree('dataset/TrashNet/validation')
base_dir = 'dataset/TrashNet_augmentedx2'
classes = ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']

train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)

def is_valid_image(file_path):
    try:
        with Image.open(file_path) as img:
            img.verify()
        return True
    except (IOError, SyntaxError) as e:
        return False

def convert_image(file_path):
    try:
        with Image.open(file_path) as img:
            if img.mode in ("P", "RGBA"):
                img = img.convert("RGBA")
            else:
                img = img.convert("RGB")
            img.save(file_path)
    except (IOError, SyntaxError) as e:
        os.remove(file_path)

for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    for filename in os.listdir(class_dir):
        file_path = os.path.join(class_dir, filename)
        if filename.lower().endswith(('.jpg', '.JPG')):
            if is_valid_image(file_path):
                convert_image(file_path)
            else:
                os.remove(file_path)
        else:
            os.remove(file_path)

split_ratio = 0.8

for cls in classes:
    class_dir = os.path.join(base_dir, cls)
    images = os.listdir(class_dir)
    random.shuffle(images)

    split_point = int(split_ratio * len(images))
    train_images = images[:split_point]
    validation_images = images[split_point:]

    for img in train_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(train_dir, cls, img)
        os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
        shutil.move(source, destination)

    for img in validation_images:
        source = os.path.join(class_dir, img)
        destination = os.path.join(validation_dir, cls, img)
        os.makedirs(os.path.join(validation_dir, cls), exist_ok=True)
        if img not in train_images:  # checking if the image has already been used
            shutil.move(source, destination)

In [5]:
base_path = '/content/dataset/TrashNet_augmentedx2'
for cls in classes:
    folder_path = os.path.join(base_path, cls)
    if os.path.exists(folder_path):
        shutil.rmtree(folder_path)

In [6]:
# Ustawienia
img_height = 224
img_width = 224
batch_size = 32
epochs = 5
# Przygotowanie danych
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_datagen = ImageDataGenerator(rescale=1./255)

In [7]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 17698 images belonging to 6 classes.
Found 6633 images belonging to 6 classes.


In [8]:
def identity_block(X, f, filters):
    """
    Implementation of the identity block

    Arguments:
    X -- input tensor
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path

    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value. You'll need this later to add back to the main path.
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [9]:
def convolutional_block(X, f, filters, s=2):
    """
    Implementation of the convolutional block

    Arguments:
    X -- input tensor
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    s -- Integer, specifying the stride to be used

    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """

    # Retrieve Filters
    F1, F2, F3 = filters

    # Save the input value
    X_shortcut = X

    # First component of main path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(s, s), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second component of main path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Third component of main path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)

    # Shortcut path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(s, s), padding='valid')(X_shortcut)
    X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation
    X = Add()([X, X_shortcut])
    X = Activation('relu')(X)

    return X

In [10]:
def ResNet50(input_shape=(224, 224, 3), classes=6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3 -> CONVBLOCK ->
    IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model instance in Keras
    """

    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    # Zero-Padding
    X = tf.keras.layers.ZeroPadding2D((3, 3))(X_input)

    # Stage 1
    X = Conv2D(64, (7, 7), strides=(2, 2), padding='valid')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)
    X = tf.keras.layers.MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f=3, filters=[64, 64, 256], s=1)
    X = identity_block(X, 3, [64, 64, 256])
    X = identity_block(X, 3, [64, 64, 256])

    # Stage 3
    X = convolutional_block(X, f=3, filters=[128, 128, 512], s=2)
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])
    X = identity_block(X, 3, [128, 128, 512])

    # Stage 4
    X = convolutional_block(X, f=3, filters=[256, 256, 1024], s=2)
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])
    X = identity_block(X, 3, [256, 256, 1024])

    # Stage 5
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], s=2)
    X = identity_block(X, 3, [512, 512, 2048])
    X = identity_block(X, 3, [512, 512, 2048])

    # AVGPOOL
    X = GlobalAveragePooling2D()(X)

    # Output layer
    X = Flatten()(X)
    X = Dense(1024, activation='relu')(X)
    X = Dropout(0.5)(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name='ResNet50')

    return model

In [11]:
# Model initialization
num_classes = len(train_generator.class_indices)
model = ResNet50(input_shape=(224, 224, 3), classes=num_classes)

# Compilation
model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, 230, 230, 3)          0         ['input_1[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv2d (Conv2D)             (None, 112, 112, 64)         9472      ['zero_padding2d[0][0]']      
                                                                                                  
 batch_normalization (Batch  (None, 112, 112, 64)         256       ['conv2d[0][0]']       

In [12]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('resnet50_custom_best_model.h5', monitor='val_loss', save_best_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)

In [ ]:
# Training the model
history_ResNet50_custom = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=5,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    callbacks=[early_stopping, model_checkpoint, reduce_lr],
    shuffle=True
)

model.save('ResNet50_custom.h5')

Epoch 1/5
  7/553 [..............................] - ETA: 3:44:08 - loss: 2.3960 - accuracy: 0.3036

In [ ]:
# Plotting training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history_ResNet50_custom.history['accuracy'], label='train_accuracy')
plt.plot(history_ResNet50_custom.history['val_accuracy'], label='val_accuracy')
plt.title('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_ResNet50_custom.history['loss'], label='train_loss')
plt.plot(history_ResNet50_custom.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.legend()

plt.show()